In [11]:
import sqlite3
import pandas as pd

def analyze_problematic_areas(db_path='../databases/dataset.db'):
    """
    ✅ ТОЛЬКО КОНСОЛЬНЫЙ ВЫВОД - БЕЗ CSV!
    """
    conn = sqlite3.connect(db_path)
    
    print("🔍 АНАЛИЗ ПРОБЛЕМНЫХ РАЙОНОВ И ЗДАНИЙ")
    print("=" * 70)
    
    # 1. КРУПНЫЕ НАРОДНЫЕ РАЙОНЫ (ТОП-10)
    print("\n🏙️  КРУПНЫЕ НАРОДНЫЕ РАЙОНЫ (ТОП-10)")
    print("-" * 40)
    
    big_folk = pd.read_sql_query("""
    SELECT 
        bfd.name AS district_name,
        COUNT(DISTINCT bb.blackout_id) as blackout_count
    FROM big_folk_districts bfd
    JOIN buildings b ON b.big_folk_district_id = bfd.id
    JOIN blackouts_buildings bb ON bb.building_id = b.id
    WHERE b.is_fake = 0
    GROUP BY bfd.id, bfd.name
    ORDER BY blackout_count DESC
    LIMIT 10
    """, conn)
    
    for idx, row in big_folk.iterrows():
        print(f"{idx+1}. {row['district_name']:25} | Отключений: {row['blackout_count']}")
    
    # 2. НАРОДНЫЕ РАЙОНЫ (ТОП-10)
    print("\n🏘️  НАРОДНЫЕ РАЙОНЫ (ТОП-10)")
    print("-" * 40)
    
    folk = pd.read_sql_query("""
    SELECT 
        fd.name AS folk_name,
        COUNT(DISTINCT bb.blackout_id) as blackout_count
    FROM folk_districts fd
    JOIN buildings b ON b.folk_district_id = fd.id
    JOIN blackouts_buildings bb ON bb.building_id = b.id
    WHERE b.is_fake = 0
    GROUP BY fd.id, fd.name
    ORDER BY blackout_count DESC
    LIMIT 10
    """, conn)
    
    for idx, row in folk.iterrows():
        print(f"{idx+1}. {row['folk_name']:25} | Отключений: {row['blackout_count']}")
    
    # 3. ПРОБЛЕМНЫЕ ЗДАНИЯ (ТОП-15)
    print("\n🏠  САМЫЕ ПРОБЛЕМНЫЕ ЗДАНИЯ (ТОП-15)")
    print("-" * 70)
    
    buildings = pd.read_sql_query("""
    SELECT 
        s.name AS street,
        b.number AS number,
        bfd.name AS big_folk_district,
        b.type,
        COUNT(DISTINCT bb.blackout_id) as blackout_count
    FROM buildings b
    JOIN streets s ON s.id = b.street_id
    JOIN big_folk_districts bfd ON bfd.id = b.big_folk_district_id
    JOIN blackouts_buildings bb ON bb.building_id = b.id
    WHERE b.is_fake = 0
    GROUP BY b.id
    ORDER BY blackout_count DESC
    LIMIT 15
    """, conn)
    
    print(f"{'№':<3} {'АДРЕС':<40} {'РАЙОН':<15} {'ТИП':<10} {'ОТКЛ'}{'':<2}")
    print("-" * 70)
    
    for idx, row in buildings.iterrows():
        address = f"{row['street']} {row['number']}"
        print(f"{idx+1:<3} {address:<40} {row['big_folk_district'][:14]:<15} "
              f"{row['type'][:9]:<10} {row['blackout_count']:<4}")
    
    # 4. ТИПЫ ОТКЛЮЧЕНИЙ (ТОП-5 РАЙОНОВ)
    print("\n⚡ ТИПЫ ОТКЛЮЧЕНИЙ ПО РАЙОНАМ (ТОП-5)")
    print("-" * 50)
    
    type_analysis = pd.read_sql_query("""
    SELECT 
        bfd.name AS district,
        bk.type,
        COUNT(DISTINCT bb.blackout_id) as count
    FROM big_folk_districts bfd
    JOIN buildings b ON b.big_folk_district_id = bfd.id
    JOIN blackouts_buildings bb ON bb.building_id = b.id
    JOIN blackouts bk ON bk.id = bb.blackout_id
    WHERE b.is_fake = 0
    GROUP BY bfd.id, bk.type
    ORDER BY count DESC
    LIMIT 15
    """, conn)
    
    current_district = ""
    for idx, row in type_analysis.iterrows():
        if row['district'] != current_district:
            if current_district:
                print()
            print(f"\n{row['district']}:")
            current_district = row['district']
        emoji = "⚡" if row['type'] == 'electricity' else "💧" if row['type'] == 'water' else "🔥"
        print(f"  {emoji} {row['type']:12} | {row['count']:3} отключений")
    
    # 5. СТАТИСТИКА
    print("\n📊 ОБЩАЯ СТАТИСТИКА")
    print("-" * 30)
    
    stats = pd.read_sql_query("""
    SELECT 
        COUNT(DISTINCT bb.blackout_id) as total_blackouts,
        COUNT(DISTINCT b.id) as affected_buildings,
        COUNT(DISTINCT bfd.id) as districts_with_blackouts
    FROM blackouts_buildings bb
    JOIN buildings b ON b.id = bb.building_id
    JOIN big_folk_districts bfd ON bfd.id = b.big_folk_district_id
    WHERE b.is_fake = 0
    """, conn).iloc[0]
    
    print(f"🔢 Всего отключений:        {stats['total_blackouts']:>6,}")
    print(f"🏠 Затронутых зданий:      {stats['affected_buildings']:>6,}")
    print(f"🏙️ Районов с отключениями: {stats['districts_with_blackouts']:>6}")
    
    conn.close()
    print("\n" + "="*70)
    print("✅ АНАЛИЗ ЗАВЕРШЕН! ВСЕ РЕЗУЛЬТАТЫ ВЫВЕДЕНЫ В КОНСОЛИ!")
    print("="*70)

# 🚀 ЗАПУСК
if __name__ == "__main__":
    analyze_problematic_areas()

🔍 АНАЛИЗ ПРОБЛЕМНЫХ РАЙОНОВ И ЗДАНИЙ

🏙️  КРУПНЫЕ НАРОДНЫЕ РАЙОНЫ (ТОП-10)
----------------------------------------
1. Чуркин                    | Отключений: 2991
2. 64, 71 микрорайоны        | Отключений: 2451
3. Вторая Речка              | Отключений: 2434
4. Центр                     | Отключений: 2382
5. Баляева                   | Отключений: 1969
6. Борисенко                 | Отключений: 1931
7. Столетие                  | Отключений: 1848
8. Третья рабочая            | Отключений: 1514
9. Луговая                   | Отключений: 1269
10. БАМ                       | Отключений: 1181

🏘️  НАРОДНЫЕ РАЙОНЫ (ТОП-10)
----------------------------------------
1. 64, 71 микрорайон         | Отключений: 2814
2. Чуркин                    | Отключений: 2153
3. Тихая                     | Отключений: 1523
4. Центр                     | Отключений: 1512
5. БАМ                       | Отключений: 1486
6. Столетие                  | Отключений: 1319
7. Окатовая                  | Отключений: 1